# 6교시 3.트랜스포머의 활용

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LayerNormalization, Dropout, Add, Input, Lambda
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 깃허브에 준비된 데이터를 가져옵니다.
!git clone https://github.com/taehojo/data.git

# CSV 파일 로드
dataframe = pd.read_csv('./data/sentiment_data.csv')

# 데이터와 라벨 추출
sentences = dataframe['sentence'].tolist()
labels = dataframe['label'].tolist()

# 임베딩 벡터 크기와 최대 문장 길이 설정
embedding_dim = 128
max_len = 10

# 토크나이저 초기화 및 텍스트를 시퀀스로 변환
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index

# 패딩을 사용하여 시퀀스 길이를 동일하게 맞춤
data = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len, padding='post')

# 데이터셋을 훈련 세트와 검증 세트로 분리
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

# 포지셔널 인코딩 함수
def get_positional_encoding(max_len, d_model):
    pos_enc = np.zeros((max_len, d_model))
    for pos in range(max_len):
        for i in range(0, d_model, 2):
            pos_enc[pos, i] = np.sin(pos / (10000 ** (2 * i / d_model)))
            if i + 1 < d_model:
                pos_enc[pos, i + 1] = np.cos(pos / (10000 ** (2 * (i + 1) / d_model)))
    return pos_enc

# 포지셔널 인코딩 생성
positional_encoding = get_positional_encoding(max_len, embedding_dim)

# 멀티헤드 어텐션 레이어
class MultiHeadSelfAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, masked=False):
        super(MultiHeadSelfAttentionLayer, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.norm = LayerNormalization()
        self.masked = masked  # 새롭게 추가된 부분

    def call(self, x):
        if self.masked:
            # 마스크드 어텐션을 적용할 경우
            batch_size = tf.shape(x)[0]  # 입력 x의 배치 크기
            seq_len = tf.shape(x)[1]     # 입력 x의 시퀀스 길이
    
            # 마스크 행렬 생성
            mask = tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)  #tf.linalg.band_part: 대각선 이전 값들을 1로 채우고 나머지는 0으로
            mask = tf.reshape(mask, (1, 1, seq_len, seq_len))  
            # 마스크 행렬의 형태를 만들어 주는 부분 -->    # 만약 [1, 1, 4, 4] 라면
                                                        # [[1, 0, 0, 0],
                                                        #  [1, 1, 0, 0],
                                                        #  [1, 1, 1, 0],
                                                        #  [1, 1, 1, 1]] 이런 형태로 변환됨
            
            mask = tf.tile(mask, [batch_size, 1, 1, 1])  # 배치 크기만큼 마스크를 반복하여 확장
            
            # 마스크 행렬을 -무한대로 변경
            mask = mask * -1e9
    
            # 마스크를 사용한 멀티헤드 어텐션
            attn_output = self.mha(query=x, value=x, key=x, attention_mask=mask)
        else:
            # 마스크 없이 멀티헤드 어텐션을 적용할 경우
            attn_output = self.mha(query=x, value=x, key=x)

        attn_output = self.norm(attn_output + x) # 잔차 연결 적용 후 레이어 정규화
        
        return attn_output

# 모델 설정
inputs = Input(shape=(max_len,))

# 1. 임베딩 레이어: 텍스트 데이터를 임베딩 벡터로 변환합니다.
embedding_layer = Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_len)
embedded_sequences = embedding_layer(inputs)

# 2. 포지셔널 인코딩 추가
embedded_sequences_with_positional_encoding = embedded_sequences + positional_encoding

# 3. 멀티헤드 어텐션 레이어 추가
attention_layer = MultiHeadSelfAttentionLayer(num_heads=8, key_dim=embedding_dim)
attention_output = attention_layer(embedded_sequences_with_positional_encoding)

# 4. 잔차 연결
attention_output_with_residual = Add()([embedded_sequences_with_positional_encoding, attention_output])

# 5. 마스크드 멀티헤드 어텐션 레이어 추가 
masked_attention_layer = MultiHeadSelfAttentionLayer(num_heads=8, key_dim=embedding_dim, masked=True)  # masked=True 적용
masked_attention_output = masked_attention_layer(attention_output_with_residual)

# 6. 잔차 연결
masked_attention_output_with_residual = Add()([attention_output_with_residual, masked_attention_output])

# 7. GlobalAveragePooling1D 레이어 추가
pooled_output = GlobalAveragePooling1D()(masked_attention_output_with_residual)

# 8. 피드 포워드 네트워크
dense_layer = Dense(128, activation='relu')(pooled_output)
dropout_layer = Dropout(0.5)(dense_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

# 모델 생성
model = Model(inputs=inputs, outputs=output_layer)

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, np.array(y_train), epochs=10, batch_size=16, validation_data=(X_val, np.array(y_val)))

# 샘플 데이터 예측
sample_texts = ["I absolutely love this!", "I can't stand this product"]
sample_sequences = tokenizer.texts_to_sequences(sample_texts)
sample_data = tf.keras.preprocessing.sequence.pad_sequences(sample_sequences, maxlen=max_len, padding='post')
predictions = model.predict(sample_data)

for i, text in enumerate(sample_texts):
    print(f"Text: {text}")
    print(f"Prediction: {'Positive' if predictions[i] > 0.5 else 'Negative'}")


Epoch 1/10
100/100 [==============================] - 3s 19ms/step - loss: 0.7871 - accuracy: 0.5281 - val_loss: 0.6386 - val_accuracy: 0.9950
Epoch 2/10
100/100 [==============================] - 2s 17ms/step - loss: 0.2017 - accuracy: 0.9400 - val_loss: 0.0031 - val_accuracy: 1.0000
Epoch 3/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0434 - accuracy: 0.9931 - val_loss: 0.0114 - val_accuracy: 0.9975
Epoch 4/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0269 - accuracy: 0.9962 - val_loss: 0.0174 - val_accuracy: 0.9975
Epoch 5/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0194 - accuracy: 0.9975 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 6/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0170 - accuracy: 0.9956 - val_loss: 0.0088 - val_accuracy: 0.9950
Epoch 7/10
100/100 [==============================] - 2s 16ms/step - loss: 0.0099 - accuracy: 0.9994 - val_loss: 0.0025 - val_accuracy: 0.9975

<참고 도표> 
데이터 특성과 모델 복잡성에 따라 마스크드 어텐션의 사용 여부를 결정해 최적의 모델을 선택하는 것이 중요합니다. 특히 마스크드 어텐션은 문장 순서가 중요한 경우에 유리합니다. 주어진 샘플의 경우, 아래와 같이 샘플 수를 줄이면 마스크드 어텐션 유무의 차이가 더욱 두드러지게 나타납니다. 이러한 요소가 주어진 데이터의 분석 성능을 향상시킬 수 있습니다.

<img src="../data/img/validation_accuracy_comparison_smooth.png" alt="Validation Accuracy Comparison" width="600"/>
